# Direct Jupyter Support
We are using a temporary directory for running this script, so we can easily remove all created files.

In [1]:
import os
from tempfile import TemporaryDirectory
import shutil

It is crucial that `nb_name` matches the name of this notebook!

In [2]:
nb_name="JupyterPyTrack.ipynb"
temp_dir = TemporaryDirectory()

shutil.copy(nb_name, temp_dir.name)
os.chdir(temp_dir.name)

Now we can define our PyTrack class as usual passing `nb_name` to the decorator.

In [3]:
from pytrack import PyTrack, parameter, result, DVCParams, PyTrackProject

@PyTrack(nb_name=nb_name)
class Stage:
    def __init__(self):
        """Class constructor

        Definition of parameters and results
        """
        self.dvc = DVCParams(outs=['calculation.txt'])

        self.n_1 = parameter(int)  # seems optional now
        self.n_2 = parameter()

        self.sum = result()
        self.dif = result()

    def __call__(self, n_1, n_2):
        """User input

        Parameters
        ----------
        n_1: First number
        n_2: Second number
        """
        self.n_1 = n_1
        self.n_2 = n_2

    def run(self):
        """Actual computation"""
        self.sum = self.n_1 + self.n_2
        self.dif = self.n_1 - self.n_2

        self.dvc.outs[0].write_text(f"{self.n_1} + {self.n_2} = {self.sum}")

Jupyter support is an experimental feature! 
Please submit issues to https://github.com/zincware/py-track.


From now on everything should behave as usual.

In [4]:
project = PyTrackProject()
project.create_dvc_repository()

project.name = "Test1"
stage = Stage()
stage(10, 20)
project.queue()

project.name = "Test2"
stage = Stage()
stage(10, 30)
project.run()

ERROR: Stage 'Stage_0' already exists in 'dvc.yaml'. Use '--force' to overwrite.



In [5]:
project.load("Test1")
stage = Stage(id_=0)
stage.sum

30

In [6]:
project.load("Test2")
stage = Stage(id_=0)
stage.sum



40

In [8]:
os.chdir('..')
temp_dir.cleanup()